Import neccessary packages

In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [ ]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '../input/plantvillage/'
width=256
height=256
depth=3

Function to convert images to array

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
import os
import cv2
import numpy as np

def convert_image_to_array(image_path, target_size=(224,224)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    return np.array(image)


directory_root = r"C:/Users/Revathi/Downloads/archive/PlantVillage/PlantVillage"


image_list = []
label_list = []

try:
    print("[INFO] Loading images ...")

    plant_folders = [f for f in os.listdir(directory_root) if f != ".DS_Store"]

    for plant_folder in plant_folders:
        plant_path = os.path.join(directory_root, plant_folder)

        disease_folders = [d for d in os.listdir(plant_path) if d != ".DS_Store"]

        for disease_folder in disease_folders:
            print(f"[INFO] Processing {disease_folder} ...")
            disease_path = os.path.join(plant_path, disease_folder)

            image_files = [img for img in os.listdir(disease_path) if img.lower().endswith(".jpg")]

            for image_file in image_files[:200]:
                image_path = os.path.join(disease_path, image_file)
                image_list.append(convert_image_to_array(image_path))
                label_list.append(disease_folder)

    print("[INFO] Image loading completed")

except Exception as e:
    print(f"Error : {e}")

Fetch images from directory

Get Size of Processed Image

In [ ]:
image_size = len(image_list)

Transform Image Labels uisng [Scikit Learn](http://scikit-learn.org/)'s LabelBinarizer

Print the classes

In [ ]:
print(label_binarizer.classes_)

In [ ]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [ ]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)

In [ ]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2,
    zoom_range=0.2,horizontal_flip=True,
    fill_mode="nearest")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

Plot the train and val curve